In [ ]:
def getListedCode(typ="twse",codeLength=4):
    import sqlite3
    sqlite_file = "list.db" #上市上櫃下市 股票代號列表 資料庫檔案
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    out = conn.execute("SELECT code, type FROM StockList WHERE type LIKE ? AND length(code) == ? ",("twse",codeLength)) 
    # 取 代號4碼 類別上市 股票代號
    codeList = []
    for i, row in enumerate(out):
        codeList.append(row[0])
    conn.close()
    print("取得股票代號總數:", len(codeList)) #codeList 為取出股票代號列表
    return codeList
print("getListedCode loaded")

codeList = getListedCode()

def getStrategyData(code,startdate,enddate,dbname="buysell.db",table="twse"):
    import sqlite3
    conn = sqlite3.connect(dbname)
    c = conn.cursor()
    out = conn.execute("SELECT * FROM twse WHERE code LIKE ? AND date between ? and ?",(code,startdate,enddate))
    dataList = []
    for i in out:
        dataList.append(i)
    conn.close()
    return dataList

dataList = getStrategyData("1101","2016-05-25","2017-05-25")
lastData = getStrategyData("1101","2017-05-24","2017-05-25","PriceVolData.db")
print("lastData:",lastData[0][6])
sellcount = 0
buycount = 0
selldata = []
buydata = []
for i in dataList:
    #print("Code:",i[1],"Open:",i[6],"Action:",i[8])
    if "sell" == i[8]:
        sellcount = sellcount + 1
        selldata.append([i[1],i[6],i[8]])
    if "buy" == i[8]:
        buycount = buycount + 1
        buydata.append([i[1],i[6],i[8]])

print("Buy:",buydata,"Sell:",selldata)
print("Buy:",buycount,"張 Sell:",sellcount,"張")

profit = 0
for buy,sell in zip(buydata,selldata):
    profit = profit + (sell[1] - buy[1])
    buydata.pop()
    selldata.pop()
    print("Profit:",profit,"Buy:",buy[1],"Sell:",sell[1])
if buycount > sellcount:
    for i in buydata:
        profit = profit + (lastData[0][6]-i[1])
        print("Profit:",profit,"Clear:",lastData[0][6],"Buy:",i[1])
print(profit)

In [1]:
def getListedCode(typ="twse",codeLength=4):
    import sqlite3
    sqlite_file = "list.db" #上市上櫃下市 股票代號列表 資料庫檔案
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    out = conn.execute("SELECT code, type FROM StockList WHERE type LIKE ? AND length(code) == ? ",("twse",codeLength)) 
    # 取 代號4碼 類別上市 股票代號
    codeList = []
    for i, row in enumerate(out):
        codeList.append(row[0])
    conn.close()
    print("取得股票代號總數:", len(codeList)) #codeList 為取出股票代號列表
    return codeList

def getStrategyData(code,startdate,enddate,dbname="buysell.db",table="twse"):
    import sqlite3
    conn = sqlite3.connect(dbname)
    c = conn.cursor()
    out = conn.execute("SELECT * FROM twse WHERE code LIKE ? AND date between ? and ?",(code,startdate,enddate))
    dataList = []
    for i in out:
        dataList.append(i)
    conn.close()
    return dataList

def getLastDate(code,dbname="PriceVolData.db",table="twse"):
    import sqlite3
    conn = sqlite3.connect(dbname)
    c = conn.cursor()
    out = conn.execute("SELECT * FROM twse WHERE code LIKE ? ORDER BY date DESC LIMIT 1",(code,))
    dataList = []
    for i in out:
        dataList.append(i)
    conn.close()
    return dataList

def getStats(dataList,lastData):
    sellcount = 0
    buycount = 0
    selldata = []
    buydata = []
    for i in dataList:
        #print("Code:",i[1],"Open:",i[6],"Action:",i[8])
        if "sell" == i[8]:
            sellcount = sellcount + 1
            selldata.append([i[1],i[6],i[8]])
        if "buy" == i[8]:
            buycount = buycount + 1
            buydata.append([i[1],i[6],i[8]])

    #print("Buy:",buydata,"Sell:",selldata)
    #print("Buy:",buycount,"張 Sell:",sellcount,"張")

    profit = 0
    win = 0
    lose = 0
    for buy,sell in zip(buydata,selldata):
        profit = profit + (sell[1] - buy[1])
        if 0 < (sell[1] - buy[1]):
            win = win + 1
        else:
            lose = lose + 1
        buydata.pop()
        selldata.pop()
        #print("Profit:",profit,"Buy:",buy[1],"Sell:",sell[1])
    if buycount > sellcount:
        #print(lastData)
        for i in buydata:
            profit = profit + (lastData[0][6]-i[1])
            if 0 < (lastData[0][6]-i[1]):
                win = win + 1
            else:
                lose = lose + 1
            #print("Profit:",profit,"Clear:",lastData[0][6],"Buy:",i[1])
    return profit,win,lose

profitList = []
profitSum = 0
winSum = 0
loseSum = 0
codeList = getListedCode()
for code in codeList:
    dataList = getStrategyData(code,"2016-05-25","2017-05-25")
    lastData = getLastDate(code,dbname="PriceVolData.db",table="twse")
    profit,win,lose = getStats(dataList,lastData)
    profitList.append([["Code:{}".format(code)],["Profit:{}".format(profit)]])
    profitSum = profitSum + profit
    winSum = winSum + win
    loseSum = loseSum + lose
print("Profit Sum:",profitSum,"Win Sum:",winSum,"Lose Sum:",loseSum)

取得股票代號總數: 913
Profit Sum: 12492.109999999988 Win Sum: 3373 Lose Sum: 2916


In [5]:
import pickle
pickle.dump(profitList, open( "profitList.pkl", "wb" ) )
profList = pickle.load( open( "profitList.pkl", "rb" ) )

#for i in profList:
#    print(i)
    
print("勝率:", winSum/(loseSum+winSum))
    
#取得股票代號總數: 913
#Profit Sum: 12492.109999999988 Win Sum: 3373 Lose Sum: 2916

[['Code:1101'], ['Profit:-9.599999999999973']]
[['Code:1102'], ['Profit:0.5000000000000036']]
[['Code:1103'], ['Profit:0.9000000000000004']]
[['Code:1104'], ['Profit:14.299999999999994']]
[['Code:1108'], ['Profit:1.9000000000000004']]
[['Code:1109'], ['Profit:-0.09999999999999964']]
[['Code:1110'], ['Profit:0.75']]
[['Code:1201'], ['Profit:6.3999999999999915']]
[['Code:1203'], ['Profit:1.0']]
[['Code:1210'], ['Profit:62.50000000000003']]
[['Code:1213'], ['Profit:3.7499999999999947']]
[['Code:1215'], ['Profit:-47.050000000000004']]
[['Code:1216'], ['Profit:38.29999999999996']]
[['Code:1217'], ['Profit:-0.71']]
[['Code:1218'], ['Profit:0']]
[['Code:1219'], ['Profit:-0.40000000000000036']]
[['Code:1220'], ['Profit:-0.3000000000000007']]
[['Code:1225'], ['Profit:-3.3999999999999986']]
[['Code:1227'], ['Profit:2.8999999999999915']]
[['Code:1229'], ['Profit:-0.3999999999999986']]
[['Code:1231'], ['Profit:12.600000000000001']]
[['Code:1232'], ['Profit:101.80000000000003']]
[['Code:1233'], ['P